In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import anndata as ad

import matplotlib.pyplot as plt
# from spida.pl import plot_categorical, plot_continuous
plt.rcParams['axes.facecolor'] = 'white'

In [ ]:
# annot_path = "/anvil/projects/x-mcb130189/qzeng/analysis/251009_merfish_methylation/Neuron.All.mC_MERFISH.Label_Transfer.Annot.tsv"
annot_path = "/anvil/projects/x-mcb130189/qzeng/analysis/251105_merfish_methylation_2/Neuron.All.mC_MERFISH.Label_Transfer.Annot.tsv"

# imputation_path = "/anvil/projects/x-mcb130189/qzeng/analysis/251009_merfish_methylation/Imputation.Subclass_Restricted.mC_MERFISH.meta.csv"
# imputation_path = "/anvil/projects/x-mcb130189/qzeng/analysis/251009_merfish_methylation/Imputation.Group_Restricted.mC_MERFISH.meta.csv"
imputation_path = "/anvil/projects/x-mcb130189/qzeng/analysis/251105_merfish_methylation_2/Imputation.Subclass_Restricted.mC_MERFISH.meta.csv"

# subclass_transfer_scores_path = "/anvil/projects/x-mcb130189/qzeng/analysis/251009_merfish_methylation/mC-MERFISH.Subclass_transfer.tsv"
group_transfer_scores_path = "/home/x-aklein2/projects/aklein/utils/methylation_imputation/mC-MERFISH.Group_transfer.tsv"
subclass_transfer_scores_path = "/anvil/projects/x-mcb130189/qzeng/analysis/251105_merfish_methylation_2/mC-MERFISH.Subclass_transfer.tsv"

In [ ]:
df = pd.read_csv(annot_path, sep="\t", index_col=0)

In [ ]:
df['Subclass_transfer'].value_counts()

# Transfer Scores

In [ ]:
ts_scores = pd.read_csv(subclass_transfer_scores_path, sep='\t', index_col=0)
dft = ts_scores.idxmax(axis=1).to_frame(name="annot").merge(df, left_index=True, right_index=True)
print("Label Transfer agreement with cluster 2 cluster annotations: %.2f%%" % ((dft['annot'] == dft['Subclass_transfer']).sum() / dft.shape[0] * 100))

In [ ]:
ft = ts_scores.reset_index().melt(id_vars = ["index"])
ft['annot'] = ft['index'].map(dft['Subclass_transfer'].to_dict())
sub_df_annots = ft.loc[ft['variable'] == ft['annot']]

In [ ]:
sub_df_annots.groupby('annot')['value'].describe()

In [ ]:
ts_scores = pd.read_csv(group_transfer_scores_path, sep='\t', index_col=0)
dft = ts_scores.idxmax(axis=1).to_frame(name="annot").merge(df, left_index=True, right_index=True)
print("Label Transfer agreement with cluster 2 cluster annotations: %.2f%%" % ((dft['annot'] == dft['Group_transfer']).sum() / dft.shape[0] * 100))

In [ ]:
ft = ts_scores.reset_index().melt(id_vars = ["index"])
ft['annot'] = ft['index'].map(dft['Group_transfer'].to_dict())
group_df_annots = ft.loc[ft['variable'] == ft['annot']]

In [ ]:
group_df_annots.groupby('annot')['value'].describe()

In [ ]:
sub_df_annots = (
    sub_df_annots.rename(columns={"annot": "subclass", "value" : "subclass_score"})
    .drop(columns=["variable"])
    .set_index("index")
)

group_df_annots = (
    group_df_annots.rename(columns={"annot": "group", "value" : "group_score"})
    .drop(columns=["variable"])
    .set_index("index")
)
# group_df_annots = df['Group_transfer'].to_frame(name="group")

In [ ]:
group_df_annots

In [ ]:
df_annots = sub_df_annots.merge(group_df_annots, left_index=True, right_index=True)

In [ ]:
# df_annots

In [ ]:
df_annots.to_csv("/home/x-aklein2/projects/aklein/BICAN/BG/data/methylation_2/annot_with_scores.csv", index=True)

In [ ]:
df_annots.head()